In [ ]:
## Module untuk menghubungkan python dengan database dan membaca query sql

import mysql.connector  # Import library untuk menghubungkan mysql dengan python
from mysql.connector import Error
import pandas as pd  # import pandas untuk menampilkan table dari database mysql


def create_server_connection(host_name, user_name, user_password):
    """ Fungsi untuk koneksi ke server.

    Parameters
    ----------
    host_name : nama host sql
    user_name : nama user sql
    user_password : password sql
    """
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password
        )
        print("MySQL connection successful")
    except Error as err:
        print("Error: {}".format(err))

    return connection


def create_database(connection, query):
    """ Fungsi untuk membuat database.
    
    Parameters
    ----------
    connection : create_server_connection(host, user, pw)
    query : query mysql untuk membuat database
    
    (CREATE DATABASE ...)
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: {err}")


def create_db_connection(host_name, user_name, user_password, db_name):
    """ Fungsi koneksi ke database.
    
    Parameters
    ----------
    host_name : nama host sql
    user_name : nama user sql
    user_password : password sql
    db_name : nama database
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name)
        print("MySQL database connection successfull")
    except Error as err:
        print(f"Error: {err}")
    return connection
        

def execute_query(connection, query):
    """ Fungsi untuk Eksekusi Query, Create, Update, Modify dan Delete data
    
    Parameters
    ----------
    connection : koneksi ke database
    query : query mysql untuk editing data
    
    (INSERT INTO ...(...) VALUES(...))
    (ALTER TABLE ... ADD / MODIFY / DROP / SET / ...)
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query berhasil dieksekusi")
    except Error as err:
        print(f"Error: {err}")
        
# Fungsi untuk eksekusi query dengan input Variable
def execute_query_var(connection, query, var):
    """ Fungsi untuk eksekusi Query create, update, delete data dengan variable.
    
    Fungsi digunakan dengan tiga parameter, hal ini bertujuan untuk dapat melakukan 
    input variable yang di buat dengan python ke dalam sql query (Umum nya variable 
    menggantikan %s pada query sql.)

    Parameters
    ----------
    connection : koneksi ke database.
    query : query mysql dengan parameter %s sebagai variable yang akan di input.
    var : variable yang di tentukan. type <tuple>, <list>, <dict>
          
    ((INSERT INTO ...(...) VALUES(%s, %s, ... )), var)

    Caution, gunakan input var* dengan type yang sudah ditentukan, selain type tersebut
    akan menghasilkan error.
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query, var)
        connection.commit()
        print("Query berhasil dieksekusi")
    except Error as err:
        print(f"Error: {err}")


def read_query(connection, query):
    """ Fungsi query untuk melakukan Penyajian dan Filtering Table dari database.
    
    Parameters
    ----------
    connection : koneksi ke database.
    query : query mysql untuk filtering dan penyajian table.
    
    (SELECT ... FROM ...)
    """
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
        return pd.DataFrame(result, columns=[field_names])
    except Error as err:
        print(f"Error: {err}")


def read_query_pencarian(connection, query, var):
    """ Fungsi query untuk melakukan penyajian dan filtering dengan variable.
    
    Parameters
    ----------
    connection : koneksi ke database.
    query : query mysql dengan parameter %s sebagai variable yang akan di input.
    var : variable yang di tentukan. type <tuple>, <list>, <dict>
    
    ((SELECT ... FROM ... WHERE ... %s), var)
    """
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query, var)
        result = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
        return pd.DataFrame(result, columns=[field_names])
    except Error as err:
        print(f"Error: {err}")

In [ ]:
## Module untuk Membuat Table

import ConnectSQL  # Import module ConnectSQL

# Parameter
user = 'root'
pw = "Megalumeng123"
host = "localhost"
db = "project_library"

# Melakukan koneksi ke server
connection = ConnectSQL.create_server_connection(host, user, pw)

# Membuat database untuk program yang akan di jalankan
create_database_query = "CREATE DATABASE project_library"
ConnectSQL.create_database(connection, create_database_query) # Melakukan eksekusi sql query

# Melakukan koneksi ke database
connection = ConnectSQL.create_db_connection(host_name=host, user_name=user, user_password=pw, db_name=db)

# Memilih database yang akan digunakan
query_use_database = """
USE project_library;
"""
ConnectSQL.execute_query(connection, query_use_database)


# Membuat table daftar user untuk peminjaman buku
query_table_daftar_user = """
CREATE TABLE daftar_user(
	id_user INT NOT NULL AUTO_INCREMENT,
    nama_user VARCHAR(50),
    tanggal_lahir DATE,
    pekerjaan VARCHAR(30),
    alamat VARCHAR(50),
    PRIMARY KEY (id_user)
);
"""

# Melakukan eksekusi query untuk membuat table daftar user
ConnectSQL.execute_query(connection, query_table_daftar_user)

# Membuat table daftar buku yang ada
query_table_daftar_buku = """
CREATE TABLE daftar_buku(
	id_buku INT PRIMARY KEY,
    nama_buku VARCHAR(30),
	kategori_buku VARCHAR(30),
    stok_buku INT 
);
"""

# Melakukan eksekusi query untuk membuat table daftar buku
ConnectSQL.execute_query(connection, query_table_daftar_buku)

# Membuat table daftar peminjaman buku
query_table_peminjam = """
CREATE TABLE peminjam(
	id_user INT,
    id_buku INT,
    nama_user VARCHAR(50),
    nama_buku VARCHAR(50),
    tanggal_pinjam DATE,
    tanggal_pengembalian DATE,
    FOREIGN KEY (id_user) REFERENCES daftar_user(id_user) ON UPDATE CASCADE,
    FOREIGN KEY (id_buku) REFERENCES daftar_buku(id_buku) ON UPDATE CASCADE
);
"""

# Melakukan eksekusi query untuk membuat table daftar peminjam
ConnectSQL.execute_query(connection, query_table_peminjam)



In [ ]:
# Fungsi Tugas

from datetime import datetime  # Import module datetime
from datetime import timedelta # Import module dateime untuk penambahan hari

import ConnectSQL  # Import module ConnectSQL

# Params
user = 'root'
pw = "Megalumeng123"
host = "localhost"
db = "project_library"


connection = ConnectSQL.create_server_connection(host, user, pw)
    
connection = ConnectSQL.create_db_connection(host_name=host, user_name=user, user_password=pw, db_name=db)


def input_user():
    """Fungsi untuk input data user.
    
    Parameter input
    ----------
    nama_user, tgl_lahir, pekerjaan, alamat
    
    Query
    -----------
    (INSERT INTO .. ( .., ..) VALUES(%s, %s), var)
    
    var = variable untuk ditambahkan kedalam query sql.
    type(var) = <tuple>
    
    Melakukan eksekusi query 
    ConnectSQL.execute_query_var
    
    """
    nama_user = str(input("Masukkan nama user:  "))
    tgl_lahir = str(input("Masukkan tanggal lahir(YYYY-MM-DD):  "))
    pekerjaan = str(input("Masukkan nama pekerjaan:  "))
    alamat = str(input("Masukkan alamat:  "))


    query_input_user = """
    INSERT INTO daftar_user(nama_user, tanggal_lahir, pekerjaan, alamat) 
    VALUES(%s,%s,%s,%s);""" 

    var_user = (nama_user, tgl_lahir, pekerjaan, alamat)
        
    ConnectSQL.execute_query_var(connection, query_input_user, var_user)

    print(f"user {nama_user} telah berhasil ditambahkan")

    
def input_buku():
    """Fungsi untuk input data buku.
    
    Parameter input
    ----------
    id_buku, nama_buku, kategori, stok_buku
    
    Query
    -----------
    (INSERT INTO .. ( .., ..) VALUES(%s, %s), var)
    
    var = variable untuk ditambahkan kedalam query sql 
    type(var) = <tuple>
    
    Melakukan eksekusi query dengan,
    ConnectSQL.execute_query_var
    
    """
    kode_buku = str(input("Masukkan kode buku:  "))
    nama_buku = str(input("Masukkan nama buku:  "))
    kategori = str(input("Masukkan kategori:  "))
    stock = str(input("Masukkan stok buku:  "))
                    
    query_input_buku = """
    INSERT INTO daftar_buku(id_buku, nama_buku, kategori_buku, stok_buku) 
    VALUES(%s,%s,%s,%s);""" 

    var_buku = (kode_buku, nama_buku, kategori, stock)
        
    ConnectSQL.execute_query_var(connection, query_input_buku, var_buku)

    print(f"Buku {nama_buku} telah di tambahkan")

    
def input_peminjam():
    """Fungsi untuk input data peminjam.
    
    Parameter input
    ----------
    id_user, id_buku, nama_peminjam, nama_buku, tgl_pinjam
    
    Query
    -----------
    (INSERT INTO .. ( .., ..) VALUES(%s, %s), var)
    
    var = variable untuk ditambahkan kedalam query sql 
    type(var) = <tuple>
    
    Melakukan eksekusi query dengan,
    ConnectSQL.execute_query_var
    """
    
    id_user = str(input("Masukkan id user:  "))
    nama_peminjam = str(input("Masukkan nama peminjam:  "))
    kode_buku = str(input("Masukkan kode buku:  "))
    nama_buku = str(input("Masukkan nama buku:  "))
    tgl_pinjam = str(input("Masukkan tanggal peminjaman:  "))

    yy, mm, dd = [int(x) for x in tgl_pinjam.split('-')]
    tgl_pengembalian = (datetime(yy, mm, dd) + timedelta(days=3)).strftime('%Y-%m-%d')
                                                                  
    query_pinjam_buku = """
    INSERT INTO peminjam(id_user, id_buku, nama_user, nama_buku, tanggal_pinjam, tanggal_pengembalian) 
    VALUES(%s,%s,%s,%s,%s,%s);""" 

    var_pinjam = (id_user, kode_buku, nama_peminjam, nama_buku, tgl_pinjam, tgl_pengembalian)
        
    ConnectSQL.execute_query_var(connection, query_pinjam_buku, var_pinjam)

    print(f"Peminjam {nama_peminjam} telah ditambahkan")

    
def daftar_user():
    """Fungsi untuk menampilkan table daftar user.
    
    Query
    -----------
    (SELECT .. FROM ... )
    
    Melakukan eksekusi query dengan,
    ConnectSQL.read_query
    """
    
    query = """
    SELECT * FROM daftar_user;
    """

    table_user = ConnectSQL.read_query(connection, query)

    display(table_user)
    

def daftar_buku():
    """Fungsi untuk menampilkan table daftar buku.
    
    Query
    -----------
    (SELECT .. FROM ... )
    
    Melakukan eksekusi query dengan,
    ConnectSQL.read_query
    """
    
    query = """
    SELECT * FROM daftar_buku;
    """
        
    table_buku = ConnectSQL.read_query(connection, query)

    display(table_buku)

    
def daftar_peminjam():
    """Fungsi untuk menampilkan table daftar peminjam.
    
    Query
    -----------
    (SELECT .. FROM ... )
    
    Melakukan eksekusi query dengan,
    ConnectSQL.read_query
    """
    
    query1 = """
    SELECT * FROM peminjam;
    """

    table_peminjam = ConnectSQL.read_query(connection, query1)  

    display(table_peminjam)

    
def cari_buku():
    nama_buku = str(input('Masukkan nama buku:  '))

    query = """
    SELECT * FROM daftar_buku WHERE nama_buku = %s;
    """
    var_pinjam = (nama_buku,)

    table_buku = ConnectSQL.read_query_pencarian(connection, query, var_pinjam)

    display(table_buku)

    
def pengembalian():
    id_user = str(input('Masukkan id peminjam:  '))
    id_buku = str(input('Masukkan id buku:  '))

    query = """
    DELETE FROM peminjam WHERE id_user = %s and id_buku = %s;
    """
    var_pengembalian = (id_user, id_buku)

    ConnectSQL.execute_query_var(connection, query, var_pengembalian)

    print('Buku sudah dikembalikan')

In [ ]:
# Untuk membuat Database dan Table Library
import buatTable

# Untuk fungsi pilihan Tugas
import inputData

finished = False;  # Boolean False

# Membuat program pilihan dengan finished bernilai False, maka apabila boolean yang diterima TRUE (not False)
# program akan terus berjalan

while not finished:
    
    # Membuat Tampilan menu awal bagi user untuk memilih tugas yang hendak dilakukan
    
    lib_manajemen = """
    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    """
    
    print(lib_manajemen);  # Menampilkan Menu
    
    pilihan = int(input("Masukkan Nomor Tugas: "))  # Melakukan input pilihan tugas
    
    if pilihan == 1:
        inputData.input_user()  # Input data user

    elif pilihan == 2:
        inputData.input_buku()  # Input data buku yang akan di stock
        
    elif pilihan == 3:
        inputData.input_peminjam()  # Input data untuk peminjam
        
    elif pilihan == 4:
        inputData.daftar_user()  # Input menampilkan daftar user
        
    elif pilihan == 5:
        inputData.daftar_buku()  # Input menampilkan daftar buku tersedia
        
    elif pilihan == 6:
        inputData.daftar_peminjam()  # Input menampilkan daftar peminjam
        
    elif pilihan == 7:  # Input untuk mencari buku
        inputData.cari_buku()
        
    elif pilihan == 8:  # Input untuk pengembalian buku
        inputData.pengembalian()
        
    elif pilihan == 9:  # Input untuk keluar dari program
        print("""
        --------------------------
        Hope you enjoy our feature
        Thank you!
        -----------
        """)
        finished = True;
    
    else:  
        print("""
        --------------------------
        Incorrect Input Value
        -----------
        """)


In [1]:
import projectlms

MySQL connection successful
Error: 1007 (HY000): Can't create database 'project_library'; database exists
MySQL database connection successfull
Query berhasil dieksekusi
Error: 1050 (42S01): Table 'daftar_user' already exists
Error: 1050 (42S01): Table 'daftar_buku' already exists
Error: 1050 (42S01): Table 'peminjam' already exists
MySQL connection successful
MySQL database connection successfull

    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 4


,id_user,nama_user,tanggal_lahir,pekerjaan,alamat
0,1,Dandi Mahendris,1998-07-07,Pegawai,Jl. pedurenan



    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 5


,id_buku,nama_buku,kategori_buku,stok_buku
0,1001,Data Cleaning,Education,10



    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 6


...


    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 7
Masukkan nama buku:  Data Cleaning


,id_buku,nama_buku,kategori_buku,stok_buku
0,1001,Data Cleaning,Education,10



    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 6


,id_user,id_buku,nama_user,nama_buku,tanggal_pinjam,tanggal_pengembalian



    ---------LIBRARY MANAGEMENT------------

        1. Pendaftaran User Baru
        2. Pendaftaran Buku Baru
        3. Peminjaman
        4. Tampilkan Daftar User
        5. Tampilkan Daftar Buku
        6. Tampilkan Daftar Peminjam
        7. Cari Buku
        8. Pengembalian
        9. Keluar
    
    ---------------------------------------
    
Masukkan Nomor Tugas: 9

        --------------------------
        Hope you enjoy our feature
        Thank you!
        -----------
        
